# 04. Self Oil Station Price Analysis

[1. 셀레니움 설치](#install-selenium) <br>
[2. 셀프 주유소가 정말 저렴하나요? - 데이터 확보](#데이터-확보) <br>
&emsp;    - [페이지 접근](#페이지-접근)<br>
&emsp;    - [팝업창 닫기](#팝업창-닫기)<br>
&emsp;    - [지역: 시/도](#지역-시도)<br>
&emsp;    - [지역: 구](#지역-구)<br>
&emsp;    - [데이터 엑셀 저장](#엑셀-저장1)<br>
&emsp;    - [서울시 모든 구의 유가 정보 엑셀 저장](#엑셀-저장2)<br>

[Github에서는 toc가 작동안함. nbviewer로 보기👀](https://nbviewer.org/github/solpinetree/ds_study/blob/main/source_code/04.%20Self%20Oil%20Station%20Price%20Analysis.ipynb)

<a id="install-selenium"></a>
## 1. 셀레니움 설치
- 윈도우, mac(intel)
    - ```conda install conda-forge::selenium```
- mac(m-)
    - ```pip install selenium```

In [9]:
from selenium import webdriver
import selenium

In [12]:
driver = webdriver.Chrome()
driver.get("https://www.naver.com")

In [13]:
driver.quit()

---

<a id="데이터-확보"></a>
## 2. 셀프 주유소가 정말 저렴하나요? - 데이터 확보
- https://www.opinet.co.kr/searRgSelect.do
- 목표 데이터
    - 브랜드
    - 가격
    - 셀프 주유 여부
    - 위치

In [24]:
from selenium import webdriver

In [35]:
from selenium.webdriver.common.by import By

<a id="페이지-접근"></a>
### 페이지 접근

In [32]:
url = "https://www.opinet.co.kr/searRgSelect.do"
driver = webdriver.Chrome()
driver.get(url)

<a id="팝업창-닫기"></a>
### 팝업창 닫기

#### 현재 떠있는 창 확인
- `driver.window_handles` 리스트의 첫번째 값이 메인 윈도우 창

In [26]:
driver.window_handles

['77DAD6F8DE511189DE98D45E7A89E069']

In [33]:
def get_main():
    pages = driver.window_handles

    for i in pages:
        if i != pages[0]:
            driver.switch_to.window(i)
            driver.close()
    
    driver.switch_to.window(pages[0])

get_main()

<a id="지역-시도"></a>
### 지역: 시/도

In [38]:
sido_list_raw = driver.find_element(By.ID, 'SIDO_NM0')
sido_list_raw.text

'            시/도\n            \n             \n             \n              서울\n             \n            \n             \n             \n              부산\n             \n            \n             \n             \n              대구\n             \n            \n             \n             \n              인천\n             \n            \n             \n             \n              광주\n             \n            \n             \n             \n              대전\n             \n            \n             \n             \n              울산\n             \n            \n             \n             \n              세종\n             \n            \n             \n              경기\n             \n             \n            \n             \n             \n              강원\n             \n            \n             \n             \n              충북\n             \n            \n             \n             \n              충남\n             \n            \n             \n             \n              

In [46]:
sido_list = sido_list_raw.find_elements(By.TAG_NAME, 'option')
len(sido_list), sido_list[0].text, sido_list[1].get_attribute("value")

(18, '시/도', '서울특별시')

In [47]:
sido_names = []

for option in sido_list:
    if option.text != '시/도':
        sido_names.append(option.get_attribute("value"))

sido_names

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원특별자치도',
 '충청북도',
 '충청남도',
 '전북특별자치도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [48]:
sido_list_raw.send_keys(sido_names[0])

<a id="지역-구"></a>
### 지역: 구

In [52]:
gu_list_raw = driver.find_element(By.ID, 'SIGUNGU_NM0')
gu_list_raw.text

'시/군/구\n강남구\n강동구\n강북구\n강서구\n관악구\n광진구\n구로구\n금천구\n노원구\n도봉구\n동대문구\n동작구\n마포구\n서대문구\n서초구\n성동구\n성북구\n송파구\n양천구\n영등포구\n용산구\n은평구\n종로구\n중구\n중랑구'

In [55]:
gu_list = gu_list_raw.find_elements(By.TAG_NAME, 'option')
gu_list[0].text, gu_list[1].text

('시/군/구', '강남구')

In [56]:
gu_names = []

for option in gu_list:
    if option.text != '시/군/구':
        gu_names.append(option.get_attribute('value'))

gu_names

['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

In [57]:
gu_list_raw.send_keys(gu_names[4])

<a id="엑셀-저장1"></a>
### 데이터 엑셀 저장

In [58]:
excel_btn = driver.find_element(By.CSS_SELECTOR, '#templ_list0 > div:nth-child(7) > div > a')
excel_btn.click()

<a id="엑셀-저장2"></a>
### 서울시 모든 구의 유가 정보 엑셀 저장

In [59]:
import time
from tqdm import tqdm_notebook

for gu in tqdm_notebook(gu_names, total = len(gu_names)):
    element = driver.find_element(By.ID, 'SIGUNGU_NM0')
    element.send_keys(gu)
    time.sleep(3)

    element_get_excel = driver.find_element(By.CSS_SELECTOR, '#templ_list0 > div:nth-child(7) > div > a')
    element_get_excel.click()
    time.sleep(3)


/var/folders/fs/9jclkm_s0_7cvyj9prcm0z6r0000gn/T/ipykernel_1349/1265379090.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for gu in tqdm_notebook(gu_names, total = len(gu_names)):


  0%|          | 0/25 [00:00<?, ?it/s]

In [60]:
driver.close()